In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# enable some thing so pyvista could work
# pv.start_xvfb()

points = [
    (0, 75, 0.1),
]

# Grid parameters
# length, width, height = 0.21, 0.31, 0.41  # dimensions of the rod. add 0.01 for di
dx, dy = 0.1, 0.1  # grid spacing in x, y, z
dt = 0.01  # time step

time = 0
end_time = 1

# Initial conditions
initial_temp = 2000
air_temp = 25

# Material properties
k = 1.5  # thermal conductivity

if dt > 1 / (2 * k * (1 / dx / dx + 1 / dy / dy)):
    dt = 1 / (2 * k * (1 / dx / dx + 1 / dy / dy))
    print(f"Adjusted dt for stability. New value dt={dt}")

# nx = ny = 0
# # Set to maximum
# for point in points:
#     x, y, t = point  # Unpack each point into x, y, z, t
#     if x > nx:
#         nx = x
#     if y > ny:
#         ny = y

# # Concidering that indices start from 0 and we need sizes
# nx += 1
# ny += 1

nx = ny = 150


u = air_temp * np.ones((nx + 2, ny + 2))

print(u.shape)


# Apply boundary conditions
# Cold end boundary condition (example: all faces at one end)
# Air temperature boundary condition (example: all faces at the opposite end)
u[0, :] = air_temp
u[:, 0] = air_temp
u[-1, :] = air_temp
u[:, -1] = air_temp


# Update temperature function for 2D
def update_temp_2d(u, dt, dx, dy, k):
    u_new = u.copy()
    for i in range(1, u.shape[0] - 1):
        for j in range(1, u.shape[1] - 1):
            u_new[i, j] = u[i, j] + k * dt * (
                (u[i + 1, j] - 2 * u[i, j] + u[i - 1, j]) / dx**2
                + (u[i, j + 1] - 2 * u[i, j] + u[i, j - 1]) / dy**2
            )

    return u_new


def apply_heat(u, x, y, r):
    u_new = u.copy()
    for i in range(1, u.shape[0] - 1):
        for j in range(1, u.shape[1] - 1):
            if ((i - x) * (i - x) + (j - y) * (j - y)) <= r * r:
                u_new[i, j] = initial_temp - (initial_temp - air_temp) * (
                    ((i - x) * (i - x) + (j - y) * (j - y)) / (r * r)
                )

    return u_new


n = 0

lastDrawN = 0


plot_temperature(u)
print(f"simulated for time {time}")

# Simulation loop
while time < end_time:
    time += dt
    n += 1
    while points and (points[0][2] < time):
        x, y, t = points.pop(0)
        u = apply_heat(u, x + 1, y + 1, 50)
    u = update_temp_2d(u, dt, dx, dy, k)
    if (n - lastDrawN) * dt > 0.1:
        lastDrawN = n
        plot_temperature(u)
        print(f"simulated for time {time}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_temperature(temperature):
    # Create a figure and axes
    fig, ax = plt.subplots()

    # Plot the temperature array as an image
    im = ax.imshow(temperature, cmap="hot")

    # Add colorbar
    cbar = fig.colorbar(im, ax=ax)

    # Set labels and title
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_title("Temperature")

    # Show the plot
    plt.show()
